In [9]:
import gdown
import pandas as pd
import os
import csv
import re

In [10]:
def download_csv():
    sheet_url = "https://docs.google.com/spreadsheets/d/1vYD9nyEoI2CEWu2UY2xpedC_4mX3TUVD-XLCjOLphiA/export?format=csv"

    # Load the CSV into a DataFrame
    df = pd.read_csv(sheet_url)
    df.to_csv("form.csv")
    # print(df)

In [11]:
# download_csv()

In [12]:
original_csv = "form.csv"
processed_csv = "processed.csv"
clothes_folder = "clothes_data"
start_index = 0
end_index = 10

In [13]:
df = pd.read_csv(original_csv)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df2 = df.drop("時間戳記", axis=1)

for index, row in df2.iterrows():
    image_url = row["照片 Picture"]  # 從表單中的照片 URL 讀取

    if re.search(r"open\?id=", image_url):
        output = f"image_{index:05}.jpg"
        df2.loc[index, "照片 Picture"] = output
df2.to_csv(processed_csv)

# check image downloaded or not

In [14]:
def download_image(image_url, path):
    file_id = image_url.split("open?id=")[-1]
    url = f"https://drive.google.com/uc?export=download&id={file_id}&confirm=t"
    gdown.download(url, path, quiet=False)

In [15]:
for index, row in df2.iterrows():
    output = f"image_{index:05}.jpg"
    path = os.path.join(clothes_folder, output)
    if not os.path.exists(path):
        try:
            image_url = df.loc[index, "照片 Picture"]
            print("download ", index, " from ", image_url)
            download_image(image_url, path)
        except Exception as e:
            print("error: ", e)